In [3]:
!pip install TurkishStemmer gensim

In [4]:
import re
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.models.phrases import Phrases, Phraser
from TurkishStemmer import TurkishStemmer
from gensim.models import Word2Vec
import multiprocessing
import pandas as pd
import string
pd.options.display.max_colwidth = 8000
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/naynco/nayn.data/master/classification_clean.csv")
df.dropna().reset_index(drop=True)

In [0]:
docs = np.array(df['Title'])
classes = np.array(df['Categories'])
df_docs = pd.DataFrame({'Document': docs, 
                        'Class': classes})
df_docs = df_docs[['Document', 'Class']]

In [7]:
df['Categories'].value_counts()

SICAK                    16889
GÜNDEM                   12983
DÜNYA                     9226
SPOR                      1967
GÜNDEM|SICAK              1135
DÜNYA|SICAK                458
SANAT                      285
DÜNYA|GÜNDEM               207
Teknoloji                  144
Video                      100
DÜNYA|SPOR                  76
DÜNYA|Teknoloji             73
SICAK|SPOR                  51
GÜNDEM|SPOR                 33
DÜNYA|SANAT                 27
SANAT|Video                 21
DÜNYA|Video                 20
GÜNDEM|SANAT                19
SPOR|Video                  17
GÜNDEM|Teknoloji            10
DÜNYA|GÜNDEM|SPOR            8
GÜNDEM|Video                 5
News                         3
DÜNYA|GÜNDEM|SICAK           3
SICAK|Teknoloji              1
SPOR|Teknoloji               1
DÜNYA|GÜNDEM|SANAT           1
DÜNYA|Teknoloji|Video        1
SICAK|Video                  1
SANAT|SICAK                  1
Name: Categories, dtype: int64

In [0]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')

def norm_doc(single_doc):
    # TR: Dokümandan belirlenen özel karakterleri ve sayıları at
    # EN: Remove special characters and numbers
    single_doc = re.sub(" \d+", " ", single_doc)
    pattern = r"[{}]".format(",.;") 
    single_doc = re.sub(pattern, "", single_doc) 
    # TR: Dokümanı küçük harflere çevir
    # EN: Convert document to lowercase
    single_doc = single_doc.lower()
    single_doc = single_doc.strip()
    # TR: Dokümanı token'larına ayır
    # EN: Tokenize documents
    tokens = WPT.tokenize(single_doc)
    # TR: Stop-word listesindeki kelimeler hariç al
    # EN: Filter out the stop-words 
    filtered_tokens = [token for token in tokens if token not in stop_word_list]
    # TR: Dokümanı tekrar oluştur
    # EN: Reconstruct the document
    single_doc = ' '.join(filtered_tokens)
    return single_doc

norm_docs = np.vectorize(norm_doc) #like magic :)
normalized_documents = norm_docs(docs)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
BoW_Vector = CountVectorizer(min_df = 0., max_df = 1.)
BoW_Matrix = BoW_Vector.fit_transform(normalized_documents)
#print (BoW_Matrix)

# TR: BoW_Vector içerisindeki tüm öznitelikleri al
# EN: Fetch al features in BoW_Vector
features = BoW_Vector.get_feature_names()
print ("features[50]:" + features[46])
print ("features[52]:" +features[48])

BoW_Matrix = BoW_Matrix.toarray()
# TR: Doküman - öznitelik matrisini göster
# EN: Print document by term matrice
BoW_df = pd.DataFrame(BoW_Matrix, columns = features)
#print(BoW_df.info())

features[50]:140
features[52]:145


In [10]:
from sklearn.decomposition import LatentDirichletAllocation
number_of_topics = 20
BoW_Matrix = BoW_Vector.fit_transform(normalized_documents)
LDA = LatentDirichletAllocation(n_components = number_of_topics, 
                                max_iter = 10, 
                                learning_offset = 50.,
                                random_state = 0,
                                learning_method = 'online').fit(BoW_Matrix)
features = BoW_Vector.get_feature_names()
for t_id, topic in enumerate(LDA.components_):
    print ("Topic %d:" % (t_id))
    print (" ".join([features[i]
                    for i in topic.argsort()[:-number_of_topics - 1:-1]]))

Topic 0:
açıklaması srail un rus dan kaybetti hayatını afrin bakan yunanistan göre aday albayrak krizi ordusu emre savaşı of ekonomik önünde
Topic 1:
suudi kaşıkçı ta arabistan kadar ankara olarak operasyonu lk hava serbest temmuz dakika gül prens ün açıldı edildi cia nı
Topic 2:
stanbul başkanı rusya eski verdi nun savaş destek tehdit terör nato cinsel nedeniyle dlib belediye vurdu bankası merkel akar verildi
Topic 3:
den geldi fazla ali ilgili açıklama lira görüştü zmir gazze uyarı yüksek hindistan yüzünden üzerine birlikte tazminat adı olsun ölüm
Topic 4:
var son kılıçdaroğlu tl bulundu la olacak in twitter davasında su davası yılın deniz hakaret anlattı yanlış buldu ceo üniversitesi
Topic 5:
nin türkiye kişi geliyor tepki eden ahmet yı facebook tutuklandı koç suriyeli sert bedelli iddia açtı maduro gazeteci kar askerlik
Topic 6:
bin açıkladı yaptı geçti olabilir soylu yılmaz önce adayı nden adalet yapan brunson hollanda güneş iş ortak çişleri bana araç
Topic 7:
erdoğan dan cumhurba